In [22]:
import pickle 
import pandas as pd
from torchsummary import summary 
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

with open('C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\data\\train_data\\df_final.pkl', 'rb') as f:
    df = pickle.load(f)

In [23]:
df = df.dropna()
df.head()

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_99_ema,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:15:00,0.401930,0.001804,2.406179,0.000063,-0.000455,-0.000250,-0.000080,0.000019,0.000373,-0.000146,...,-0.000228,-0.000111,-0.000055,-41.318663,-47.808444,-0.039246,0.444260,-0.111538,-4.903050,6.489781
2022-11-14 20:16:00,-0.322941,0.000225,-0.875063,0.000342,-0.000482,-0.000243,-0.000081,0.000020,0.000336,-0.000118,...,-0.000218,-0.000109,-0.000054,-37.196986,-45.686152,-0.031542,0.447616,-0.048051,-1.803474,8.489166
2022-11-14 20:17:00,0.047696,0.001045,3.640008,0.000262,-0.000404,-0.000231,-0.000075,0.000020,0.000513,-0.000028,...,-0.000190,-0.000101,-0.000052,-32.190122,-42.986946,-0.023471,0.427829,-0.003950,-1.147692,10.796824
2022-11-14 20:18:00,0.751147,0.000294,-0.718221,0.000422,-0.000253,-0.000242,-0.000069,0.000020,0.000459,-0.000003,...,-0.000180,-0.000098,-0.000051,-27.518817,-39.893321,-0.009338,0.437425,0.011163,14.748759,12.374503
2022-11-14 20:19:00,-0.154670,0.000703,1.388568,0.000675,-0.000139,-0.000222,-0.000065,0.000020,0.000520,0.000051,...,-0.000160,-0.000093,-0.000050,-22.633165,-36.441289,0.002610,0.524746,0.003284,-1.205912,13.808125


In [24]:
df['rolling_volatility'] = df['mid_price'].rolling(window=20).std()

In [26]:
df = df.dropna()

In [27]:
df['mid_price']

,volume,mid_price,log_returns,7min_MA,25min_MA,99min_MA,300min_MA,900min_MA,7_ema,25_ema,...,log_300_ema,log_900_ema,MACD,Signal_Line,obv,rsi,vma,vol_pct,MACD-SIGNAL,rolling_volatility
time,,,,,,,,,,,,,,,,,,,,,
2022-11-14 20:34:00,0.030846,0.000360,-1.341817,-0.000007,0.000191,-0.000212,-0.000044,0.000025,-0.000029,0.000025,...,-0.000082,-0.000047,2.357736,-1.427517,0.021276,0.547739,-0.006884,-0.950316,3.785253,0.000624
2022-11-14 20:35:00,0.473259,-0.000312,-1.868468,0.000033,0.000114,-0.000222,-0.000042,0.000026,-0.000100,-0.000001,...,-0.000084,-0.000048,1.933080,-0.755398,0.010957,0.552191,-0.006604,14.342421,2.688478,0.000511
2022-11-14 20:36:00,-0.530624,0.000345,-2.104210,-0.000010,0.000161,-0.000208,-0.000043,0.000026,0.000012,0.000025,...,-0.000081,-0.000047,2.026263,-0.199066,0.015801,0.581805,-0.017948,-2.121213,2.225328,0.000513
2022-11-14 20:37:00,0.345804,0.000882,1.554977,0.000073,0.000239,-0.000185,-0.000046,0.000026,0.000229,0.000091,...,-0.000074,-0.000045,3.221306,0.485009,0.022319,0.602201,-0.009922,-1.651693,2.736298,0.000498
2022-11-14 20:38:00,0.485334,0.000429,-0.513033,0.000072,0.000251,-0.000193,-0.000043,0.000025,0.000279,0.000117,...,-0.000071,-0.000044,4.678889,1.323785,0.032000,0.592709,-0.028731,0.403494,3.355105,0.000502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-14 05:11:00,-0.623391,0.000028,-1.249393,0.000038,0.000018,0.000014,0.000012,-0.000010,0.000015,0.000036,...,0.000011,-0.000004,11.503282,12.163212,-1.683754,0.756277,-0.133589,-1.217872,-0.659931,0.000209
2023-11-14 05:12:00,1.616337,0.000112,3.064072,0.000054,0.000020,0.000014,0.000008,-0.000010,0.000039,0.000042,...,0.000012,-0.000004,11.388726,12.008315,-1.683466,0.750267,-0.027362,-3.592813,-0.619589,0.000185
2023-11-14 05:13:00,2.921293,0.000155,0.376187,0.000043,0.000021,0.000016,0.000011,-0.000010,0.000068,0.000051,...,0.000013,-0.000003,11.619901,11.930632,-1.684597,0.737423,0.091883,0.807355,-0.310732,0.000173


In [11]:
threshold = 0.0001
target = pd.cut(df['mid_price'], bins=[-float('inf'), -threshold, threshold, float('inf')], labels=[0, 1, 2])
target.value_counts()

1    184001
2    170698
0    169418
Name: mid_price, dtype: int64

In [12]:
feature = df

# 주어진 데이터프레임
sequence_length = 64
n_samples = len(df) - sequence_length + 1

# Extract the data as a NumPy array
data_array = df.to_numpy()

# Create an empty array to store the input sequences
input_sequences = np.empty((n_samples, (df.shape[1]), sequence_length))

for i in tqdm(range(n_samples)):
    input_seq = data_array[i:i + sequence_length, :].T  # Transpose to have (24, 60) shape
    input_sequences[i] = input_seq

print(input_sequences.shape)

100%|██████████| 524054/524054 [00:09<00:00, 54242.03it/s] 

(524054, 30, 64)


In [13]:
X = input_sequences[:-1]
y = target.shift(-1)[63:].dropna()

X.shape, y.shape

((524053, 30, 64), (524053,))

In [14]:
# 피처와 타겟 데이터 생성 (예시 데이터)
num_samples = 524052
num_channels = 1
num_classes = 3

feature = np.array(X[-100000:], dtype=np.float32)
target = np.array(y[-100000:], dtype=np.float32)

print(feature.shape, target.shape)

(100000, 30, 64) (100000,)


In [15]:
from classifier import Classifier 

dataset = TensorDataset(torch.tensor(feature, dtype=torch.float32), torch.tensor(target, dtype=torch.int64))
batch_size = 256
dataloader = DataLoader(dataset, batch_size=batch_size)

print("model loading...")
# 모델 초기화
classifier_model = Classifier(input_channels=num_channels, num_classes=3)

print("optimizer loading...")
# 손실 함수 및 최적화기 설정
criterion = nn.CrossEntropyLoss()

learning_rate = 0.001

optimizer = optim.Adam(classifier_model.parameters(), lr=learning_rate)

model loading...
optimizer loading...


In [18]:
# 학습 루프
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch in dataloader:
        inputs, labels = batch

        # 모델 예측
        outputs = classifier_model(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파 및 가중치 갱신
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # 정확도 계산
        _, predicted_labels = torch.max(outputs, 1)
        correct_predictions += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    # 에폭마다 평균 손실 및 정확도 출력
    average_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_samples * 100
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%")

print("Training finished.")

KeyboardInterrupt: 

In [ ]:
# torch.save(classifier_model, "C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\bitcoin_scalping_bot\\version_4\\classifier.pt")

In [19]:
model = torch.load("C:\\Users\\user\\Documents\\GitHub\\crypto-scalping-RL-Agent\\bitcoin_scalping_bot\\version_4\\classifier.pt")

In [20]:
# Print summary for Conv2d layers
print("Conv2d_1:")
print(model.conv2d_1)
print("\nConv2d_2:")
print(model.conv2d_2)
print("\nConv2d_3:")
print(model.conv2d_3)

# Print summary for Linear layers
print("\nFC1:")
print(model.fc1)
print("\nFC2:")
print(model.fc2)


Conv2d_1:
Conv2d(1, 8, kernel_size=(12, 2), stride=(12, 2))

Conv2d_2:
Conv2d(8, 16, kernel_size=(1, 17), stride=(1, 1))

Conv2d_3:
Conv2d(16, 32, kernel_size=(1, 9), stride=(1, 1))

FC1:
Linear(in_features=64, out_features=32, bias=True)

FC2:
Linear(in_features=32, out_features=3, bias=True)


In [21]:
model

Classifier(
  (conv2d_1): Conv2d(1, 8, kernel_size=(12, 2), stride=(12, 2))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_2): Conv2d(8, 16, kernel_size=(1, 17), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2d_3): Conv2d(16, 32, kernel_size=(1, 9), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm1): LSTM(512, 64)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=3, bias=True)
)